## Load the following dependencies

In [2]:
import os
import cv2
import json
import random
import re
import numpy as np
from PIL import Image
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from tensorflow.keras.layers import (
    Dense, Conv2D, MaxPool2D, 
    Flatten, Dropout, BatchNormalization,
)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from sklearn.model_selection import train_test_split
from keras import backend as K


## Coordinate appointing

In [3]:

# Groups coordinates of Waldo with the image
def coord_Data_Appointing(dataset, coord_file, w, binary, hasFile):
    # Opens the coordinates file
    if(hasFile == True):
        with open(coord_file, 'r') as f:
            coords = json.load(f)
    imgData_pairs = []
    imgDirect = os.path.dirname(dataset[0][0])
    # Retrieves the image name and the coordinates
    # Appends them together and preps them for coord appointing
    for img in dataset:
        imgPath = img[0]
        imgName = os.path.basename(imgPath)
        if(imgName != '.DS_Store'):
            pattern = r'\d+'
            digits = re.findall(pattern, imgName)
            if len(digits[0]) == 1:
                digits[0] = '0' + digits[0]
            imgData_pairs.append((imgName, digits)) 

    imgData_pairs = sorted(imgData_pairs, key=lambda x: x[1])

    # Coordinates are assigned to the image
    for img in imgData_pairs:
        imgName = img[0]
        # Get coords
        xPos = -w
        yPos = -w
        if hasFile == True:
            # issue?
            for item in coords[str(int(w))][str(int(img[1][0]))]:
                if(item["x"] == (img[1][1]) and item["y"] == (img[1][2])):
                    xPos = item["x_px"] / 64
                    yPos = item["y_px"] / 64
                    break

        xyCoords = [xPos, yPos]
        dataIndx = dataset.index((imgDirect + '/' + imgName, binary))
        
        temp = list(dataset[dataIndx])
        temp.append(xyCoords)
        dataset[dataIndx] = tuple(temp)        


## Splitting the data correctly

In [4]:
# Splits the data into training, validation, and testing sets
# STATUS: COMPLETE?
def data_splitting(dir1, dir1_1, dir1_2, dir2, w):
    # Each directory (waldo, not waldo) are given a binary classification
    sec1 = [(os.path.join(dir1, f), 0) for f in os.listdir(dir1)]
    sec1_1 = [(os.path.join(dir1_1, f), 0) for f in os.listdir(dir1_1)]
    sec1_2 = [(os.path.join(dir1_2, f), 0) for f in os.listdir(dir1_2)]
    sec2 = [(os.path.join(dir2, f), 1) for f in os.listdir(dir2)]

    coordsFile = os.path.join('Hey-Waldo-master', 'data.json')
    print(type(sec1))
    coord_Data_Appointing(sec1, coordsFile, w, 0, True)
    # =============
    # EXPERIMENTAL
    coord_Data_Appointing(sec1_1, coordsFile, w, 0, True)
    coord_Data_Appointing(sec1_2, coordsFile, w, 0, True)
    # EXPERIMENTAL
    # =============
    coord_Data_Appointing(sec2, coordsFile, w, 1, False)
    
    # Combines both sets and splits them into training, validation, and testing sets
    # 70% training, 20% validation, 10% testing
    #dataset = sec1 + sec1_1 + sec1_2 + sec2
    container = []
    for j in range(41):
        container = container + sec1 + sec1_1 + sec1_2

    dataset = container + sec2
    
    #dataset = sec1 + sec2
    train, test = train_test_split(dataset, test_size=0.3, random_state=42)
    valid, test = train_test_split(test, test_size=0.33, random_state=42)
    return train, valid, test


## Adjust the data to the correct input for the model

In [5]:
# Adjusts the data to the appropriate size
def dataAdjusting(imgDataset, width, height):
    imgs = []
    coords = []
    for imgPath in imgDataset:
        if(os.path.basename(imgPath[0]) != '.DS_Store'):
            img = cv2.imread(imgPath[0])
            img = cv2.resize(img, (width, height))
            img = img / 255.0
            imgs.append(img)

            normCoords = [(imgPath[2][0])/width, (imgPath[2][1])/height]
            coords.append(normCoords)

    imgs = np.array(imgs)
    #coords = (np.array(coords))/(width * height)
    coords = (np.array(coords))

    return imgs, coords

def predictions(imgPath, width, height, model):
    image = image_utils.load_img(imgPath, target_size=(width, height))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,width,height,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds


In [27]:
#######################################################################
#######################################################################
# START OF PROGRAM
#######################################################################
#######################################################################

# Combining the data and splitting into the appropriate sizes
# STATUS: INCOMPLETE
waldo_dir = os.path.join('Hey-Waldo-master', '64', 'waldo')
waldo_bw_dir = os.path.join('Hey-Waldo-master', '64-bw', 'waldo')
waldo_grey_dir = os.path.join('Hey-Waldo-master', '64-gray', 'waldo')
not_waldo_dir = os.path.join('Hey-Waldo-master', '64-copy', 'notwaldo')

# Current step is in the works
trainingSet, validSet, testingSet = data_splitting(waldo_dir, waldo_bw_dir, waldo_grey_dir, not_waldo_dir, 64)

print(f"Training Set: {len(trainingSet)}")
print(f"Validation Set: {len(validSet)}")
print(f"Testing Set: {len(testingSet)}")


#######################################################################
# Gets the image size parameters
# STATUS: INCOMPLETE
imgPath = os.path.join('Hey-Waldo-master', '64', 'waldo', '1_1_1.jpg')
imgPath = testingSet[0][0]

img = Image.open(imgPath)
width = img.width
height = img.height
imageSize = (width, height)

xTrain, yTrain = dataAdjusting(trainingSet, width, height)
xValid, yValid = dataAdjusting(validSet, width, height)
xTest, yTest = dataAdjusting(testingSet, width, height)

imgGen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")
imgGen.fit(xTrain)


<class 'list'>
Training Set: 3322
Validation Set: 954
Testing Set: 470


In [48]:
print(len(testingSet))
testingSet[468][2]

470


[6.0, 3.0]

## Modified CNN model & Pretrained CNN model implementation

In [49]:
#######################################################################
# CNN Model based from NVIDIA Code with modificiations

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3)))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPool2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='linear'))  # Two output neurons for the x and y coordinates

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xTrain, yTrain , validation_data=(xValid, yValid), 
          epochs=10, batch_size=4)

# ADDING A NEW MODEL
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(width, height, 3),
    include_top=False
)
base_model.trainable = True

inputs = keras.Input(shape=(width, height, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(2)(x)
vgg16_model = keras.Model(inputs, outputs)

vgg16_model.compile(optimizer='adam', loss='mean_squared_error')
vgg16_model.fit(xTrain, yTrain , validation_data=(xValid, yValid), 
          epochs=10, batch_size=4)


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 46s 55ms/step - loss: 0.0595 - val_loss: 0.0103
Epoch 2/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 46s 55ms/step - loss: 0.0069 - val_loss: 0.0049
Epoch 3/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 50s 60ms/step - loss: 0.0035 - val_loss: 0.0040
Epoch 4/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 55s 66ms/step - loss: 0.0016 - val_loss: 0.0029
Epoch 5/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 48s 58ms/step - loss: 8.2924e-04 - val_loss: 0.0032
Epoch 6/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 47s 57ms/step - loss: 9.6736e-04 - val_loss: 0.0024
Epoch 7/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 51s 62ms/step - loss: 7.2731e-04 - val_loss: 0.0024
Epoch 8/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 55s 66ms/step - loss: 6.2865e-04 - val_loss: 0.0020
Epoch 9/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 55s 66ms/step - loss: 4.8964e-04 - val_loss: 0.0019
Epoch 10/10
831/831 ━━━━━━━━━━━━━━━━━━━━ 57s 69ms/step - loss: 5.1322e-04 - val_loss: 0.0025


## Testing with the Modified model

In [62]:
# TESTING WITH TWO MODELS
xValError = 0
yValError = 0
x_errorPercentage = 0
y_errorPercentage = 0
print("\n\n=== === ===\nNOW TESTING WITH WALDO - Scratch Model")
for number in range(len(testingSet)):
    imgResults = predictions(testingSet[number][0], width, height, model)
    print(f"Image: {testingSet[number][0]} \Actual Results: {testingSet[number][2]}")
    print(f"Predicted Results: {imgResults}\n")

    xValError = imgResults[0][0] - testingSet[number][2][0]
    yValError = imgResults[0][1] - testingSet[number][2][1]

    x_errorPercentage += (xValError/testingSet[number][2][0]) * 100
    y_errorPercentage += (yValError/testingSet[number][2][1]) * 100


print(f"X Error: {x_errorPercentage/len(testingSet)}")
print(f"Y Error: {y_errorPercentage/len(testingSet)}")



=== === ===
NOW TESTING WITH WALDO - Scratch Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-bw/waldo/7_12_5.jpg 
Predicted Results: [12.0, 5.0]
Predicted Results: [[11.976232   2.6084356]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-bw/waldo/4_2_12.jpg 
Predicted Results: [2.0, 12.0]
Predicted Results: [[6.949501  0.7677355]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/18_15_6.jpg 
Predicted Results: [15.0, 6.0]
Predicted Results: [[-0.5483464 -2.1481595]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-gray/waldo/10_15_4.jpg 
Predicted Results: [15.0, 4.0]
Predicted Results: [[8.4243765 4.0091696]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/13_5_2.jpg 
Predicted Results: [5.0, 2.0]
Predicted Results: [[-0.13280642 -1.482341  ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/12_2_2.jpg 
Predicted Results: [2.0, 2.0]
Predicted Results: [[-3.855120

/var/folders/fr/1q2qc89926lc3r6mb8jjx_tr0000gn/T/ipykernel_55496/3662526275.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  x_errorPercentage += (xValError/testingSet[number][2][0]) * 100
/var/folders/fr/1q2qc89926lc3r6mb8jjx_tr0000gn/T/ipykernel_55496/3662526275.py:15: RuntimeWarning: invalid value encountered in scalar add
  x_errorPercentage += (xValError/testingSet[number][2][0]) * 100


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-gray/waldo/1_4_6.jpg 
Predicted Results: [4.0, 6.0]
Predicted Results: [[5.3260612 3.1218555]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/2_7_3.jpg 
Predicted Results: [7.0, 3.0]
Predicted Results: [[1.3542967 2.944714 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/2_7_3.jpg 
Predicted Results: [7.0, 3.0]
Predicted Results: [[1.3542967 2.944714 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-bw/waldo/5_7_3.jpg 
Predicted Results: [7.0, 3.0]
Predicted Results: [[ 6.0942698 -3.5112424]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64-bw/waldo/2_2_5.jpg 
Predicted Results: [2.0, 5.0]
Predicted Results: [[ 6.6933675 -0.5889721]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-Waldo-master/64/waldo/11_6_11.jpg 
Predicted Results: [6.0, 11.0]
Predicted Results: [[ 5.909622   -0.07680488]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Image: Hey-

## Testing with the Pretrained Model

In [64]:
print("\n\n=== === ===\nNOW TESTING WITH WALDO - Pretrained Model")
xValError = 0
yValError = 0
x_errorPercentage = 0
y_errorPercentage = 0
for number in range(len(testingSet)):
    imgResults = predictions(testingSet[number][0], width, height, vgg16_model)
    print(f"\nImage: {testingSet[number][0]} \nActual Results: {testingSet[number][2]}")
    print(f"Predicted Results: {imgResults}\n")
    xValError = imgResults[0][0] - testingSet[number][2][0]
    yValError = imgResults[0][1] - testingSet[number][2][1]

    x_errorPercentage += (xValError/testingSet[number][2][0]) * 100
    y_errorPercentage += (yValError/testingSet[number][2][1]) * 100


print(f"X Error: {x_errorPercentage/len(testingSet)}")
print(f"Y Error: {y_errorPercentage/len(testingSet)}")



=== === ===
NOW TESTING WITH WALDO - Pretrained Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Image: Hey-Waldo-master/64-bw/waldo/7_12_5.jpg 
Actual Results: [12.0, 5.0]
Predicted Results: [[  3.7519155 -28.917213 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

Image: Hey-Waldo-master/64-bw/waldo/4_2_12.jpg 
Actual Results: [2.0, 12.0]
Predicted Results: [[28.414469  -3.3929257]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step

Image: Hey-Waldo-master/64/waldo/18_15_6.jpg 
Actual Results: [15.0, 6.0]
Predicted Results: [[-7.824945  -3.7869995]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

Image: Hey-Waldo-master/64-gray/waldo/10_15_4.jpg 
Actual Results: [15.0, 4.0]
Predicted Results: [[28.339859 -4.547207]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Image: Hey-Waldo-master/64/waldo/13_5_2.jpg 
Actual Results: [5.0, 2.0]
Predicted Results: [[ -1.3438125 -20.836895 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Image: Hey-Waldo-master/64/waldo/12_2_2.jpg 
Actual Results: [2.0, 2.0]
Predicted Results: [[ 0.52303

/var/folders/fr/1q2qc89926lc3r6mb8jjx_tr0000gn/T/ipykernel_55496/3036007101.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  x_errorPercentage += (xValError/testingSet[number][2][0]) * 100


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64-bw/waldo/9_9_5.jpg 
Actual Results: [9.0, 5.0]
Predicted Results: [[14.581773  -5.1906223]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64-gray/waldo/4_2_11.jpg 
Actual Results: [2.0, 11.0]
Predicted Results: [[31.69754 34.32759]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64-bw/waldo/19_0_7.jpg 
Actual Results: [0.0, 7.0]
Predicted Results: [[17.134977   1.5754533]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64-bw/waldo/9_0_11.jpg 
Actual Results: [0.0, 11.0]
Predicted Results: [[-0.93345135 -1.3815842 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64-gray/waldo/1_4_6.jpg 
Actual Results: [4.0, 6.0]
Predicted Results: [[ 36.64828  -22.286839]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64/waldo/2_7_3.jpg 
Actual Results: [7.0, 3.0]
Predicted Results: [[-3.956001 -4.381799]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image

/var/folders/fr/1q2qc89926lc3r6mb8jjx_tr0000gn/T/ipykernel_55496/3036007101.py:13: RuntimeWarning: invalid value encountered in scalar add
  x_errorPercentage += (xValError/testingSet[number][2][0]) * 100



Image: Hey-Waldo-master/64-bw/waldo/5_7_3.jpg 
Actual Results: [7.0, 3.0]
Predicted Results: [[ -2.8112414 -19.725428 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Image: Hey-Waldo-master/64-bw/waldo/2_2_5.jpg 
Actual Results: [2.0, 5.0]
Predicted Results: [[11.175315  -2.2395463]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Image: Hey-Waldo-master/64/waldo/11_6_11.jpg 
Actual Results: [6.0, 11.0]
Predicted Results: [[ -3.5849242 -16.537354 ]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Image: Hey-Waldo-master/64/waldo/13_2_11.jpg 
Actual Results: [2.0, 11.0]
Predicted Results: [[-57.37982  -31.610353]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Image: Hey-Waldo-master/64-gray/waldo/5_7_3.jpg 
Actual Results: [7.0, 3.0]
Predicted Results: [[31.165857 14.265556]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

Image: Hey-Waldo-master/64/waldo/17_13_5.jpg 
Actual Results: [13.0, 5.0]
Predicted Results: [[10.69884    -0.94740134]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Image: Hey-Waldo-master/64/waldo/14_1_1